<a href="https://colab.research.google.com/github/sunbooshi/1Click-Bert-VITS2/blob/main/1ClickBertVITS2.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 环境准备



In [ ]:
#@title 安装Bert-VITS2
!git clone https://github.com/fishaudio/Bert-VITS2
%cd Bert-VITS2
!git checkout aae043f0e39fe1dc85098ce5df56c4fa6243a1dd
%pip install -r requirements.txt
%cd /content/

In [ ]:
#@title 安装audio-slicer
!git clone https://github.com/openvpi/audio-slicer.git audio_slicer

In [ ]:
#@title 安装whisper
!pip install git+https://github.com/openai/whisper.git
!pip install torchaudio
# 下载模型
!wget -P /content/whisper/ https://openaipublic.azureedge.net/main/whisper/models/345ae4da62f9b3d59415adc60127b97c714f32e89e936602e85993674d08dcb1/medium.pt

In [ ]:
#@title 安装onnxruntime
!pip install onnx onnxruntime

In [ ]:
#@title 安装ffmpeg
import os, uuid, re, IPython
import ipywidgets as widgets
import time

from glob import glob
from google.colab import output, drive

from IPython.display import clear_output
import os, sys, urllib.request
HOME = os.path.expanduser("~")
pathDoneCMD = f'{HOME}/doneCMD.sh'
if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/yunooooo/gcct/master/res/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")

from ttmg import (
    loadingAn,
    textAn,
)

loadingAn(name="lds")
textAn("Cloning Repositories...", ty='twg')
!git clone https://github.com/XniceCraft/ffmpeg-colab.git
!chmod 755 ./ffmpeg-colab/install
textAn("Installing FFmpeg...", ty='twg')
!./ffmpeg-colab/install
clear_output()
print('Installation finished!')
!rm -fr /content/ffmpeg-colab
!ffmpeg -version

In [ ]:
#@title 准备目录
%cd /content/Bert-VITS2/
import os
def mkdir(path):
  if os.path.exists(path):
    print(f'{path} exists')
  else:
    os.mkdir(path)
data_dir = 'Data'
mkdir(data_dir)
speaker = "paimeng" # @param {type:"string"}
speaker_dir = os.path.join(data_dir, speaker)
mkdir(speaker_dir)
audios = os.path.join(speaker_dir, 'audios')
mkdir(audios)
audios_raw = os.path.join(audios, 'raw')
mkdir(audios_raw)
audios_wavs = os.path.join(audios, 'wavs')
mkdir(audios_wavs)
filelists = os.path.join(speaker_dir, 'filelists')
mkdir(filelists)
input_dir = "inputs"
mkdir(input_dir)
models_dir = os.path.join(speaker_dir, 'models')
mkdir(models_dir)
%env NLTK_DATA=/content/nltk_data
!mkdir /content/nltk_data

# 模型下载

In [ ]:
#@title 下载模型
!wget -P slm/wavlm-base-plus/ https://huggingface.co/microsoft/wavlm-base-plus/resolve/main/pytorch_model.bin
!wget -P bert/chinese-roberta-wwm-ext-large/ https://huggingface.co/hfl/chinese-roberta-wwm-ext-large/resolve/main/pytorch_model.bin
!wget -P bert/deberta-v3-large/ https://huggingface.co/microsoft/deberta-v3-large/resolve/main/pytorch_model.bin
!wget -P bert/deberta-v3-large/ https://huggingface.co/microsoft/deberta-v3-large/resolve/main/pytorch_model.generator.bin
!wget -P bert/deberta-v3-large/ https://huggingface.co/microsoft/deberta-v3-large/resolve/main/spm.model
!wget -P bert/deberta-v2-large-japanese-char-wwm/ https://huggingface.co/ku-nlp/deberta-v2-large-japanese-char-wwm/resolve/main/pytorch_model.bin

In [ ]:
# @title 训练底模
!wget -P $models_dir https://huggingface.co/OedoSoldier/Bert-VITS2-2.3/resolve/main/DUR_0.pth
!wget -P $models_dir https://huggingface.co/OedoSoldier/Bert-VITS2-2.3/resolve/main/D_0.pth
!wget -P $models_dir https://huggingface.co/OedoSoldier/Bert-VITS2-2.3/resolve/main/G_0.pth
!wget -P $models_dir https://huggingface.co/OedoSoldier/Bert-VITS2-2.3/resolve/main/WD_0.pth

# 准备训练数据

## 上传训练数据

In [ ]:
# @title ### 直接上传
import shutil
from google.colab import files

uploaded = files.upload()

for f in uploaded.keys():
  shutil.move(f, input_dir)

### 从GoogleDrive 复制

In [ ]:
# @title #### 挂载GoogleDrive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# @title #### 复制文件
import shutil
# @markdown  输入要复制的文件路径：
drive_path = "/content/drive/MyDrive/data/paimeng.wav" # @param {type:"string"}
if os.path.isdir(drive_path):
  !cp {drive_path}*.wav {input_dir}
else:
  !cp {drive_path} {input_dir}

## 数据处理

In [ ]:
# @title ### 音频分割
import librosa  # Optional. Use any library you like to read audio files.
import soundfile  # Optional. Use any library you like to write audio files.
import os
import sys
sys.path.append('/content/audio_slicer')
from slicer2 import Slicer

def slice_audio(in_file, out_path, out_name, base_index=0):
    audio, sr = librosa.load(in_file, sr=None, mono=False)  # Load an audio file with librosa.
    slicer = Slicer(
        sr=sr,
        threshold=-40,
        min_length=2000,
        min_interval=300,
        hop_size=10,
        max_sil_kept=500
    )
    chunks = slicer.slice(audio)
    total = 0
    for i, chunk in enumerate(chunks):
        total = total + 1
        if len(chunk.shape) > 1:
            chunk = chunk.T  # Swap axes if the audio is stereo.
        out_file = os.path.join(out_path, '%s_%03d.wav' % (out_name, base_index+i))
        soundfile.write(out_file, chunk, sr)  # Save sliced audio files with soundfile.

    return total

def slice_dir(input_dir, output_dir, model_name):
    # 列出当前工作目录中的所有文件和文件夹
    files = os.listdir(input_dir)

    base_index = 0
    # 筛选出所有以 ".wav" 结尾的文件
    wav_files = [f for f in files if f.endswith(".wav")]

    # 打印所有 wav 文件的名称
    for f in wav_files:
        base_index = base_index + slice_audio(os.path.join(input_dir, f), output_dir, model_name, base_index)


if len(input_dir)>0 and len(audios_raw)>0 and len(speaker)>0:
  slice_dir(input_dir, audios_raw, speaker)
else:
  print("error input")

print("分割完成")

In [ ]:
# @title ### 音频标注

import whisper
import os
import json
import torchaudio
import argparse
import torch

lang2token = {
            'zh': "ZH|",
            'ja': "JP|",
            "en": "EN|",
        }

def transcribe(audio_path):
    # load audio and pad/trim it to fit 30 seconds
    audio = whisper.load_audio(audio_path)
    audio = whisper.pad_or_trim(audio)

    # make log-Mel spectrogram and move to the same device as the model
    mel = whisper.log_mel_spectrogram(audio).to(model.device)

    # detect the spoken language
    _, probs = model.detect_language(mel)
    print(f"Detected language: {max(probs, key=probs.get)}")
    lang = max(probs, key=probs.get)
    # decode the audio
    options = whisper.DecodingOptions(beam_size=5)
    result = whisper.decode(model, mel, options)

    # print the recognized text
    print(result.text)
    return lang, result.text


# assert (torch.cuda.is_available()), "Please enable GPU in order to run Whisper!"
model = whisper.load_model("medium", download_root="/content/whisper")
speaker_annos = []
total_files = sum([len(files) for r, d, files in os.walk(audios_raw)])
processed_files = 0

for i, wavfile in enumerate(list(os.walk(audios_raw))[0][2]):
    # try to load file as audio
    try:
        lang, text = transcribe(os.path.join(audios_raw, wavfile))
        if lang not in list(lang2token.keys()):
            print(f"{lang} not supported, ignoring\n")
            continue

        text = f"{audios_wavs}/{wavfile}|" + f"{speaker}|" +lang2token[lang] + text + "\n"
        speaker_annos.append(text)

        processed_files += 1
        print(f"Processed: {processed_files}/{total_files}")
    except Exception as e:
        print(e)
        continue

if len(speaker_annos) == 0:
    print("Warning: no short audios found, this IS expected if you have only uploaded long audios, videos or video links.")
    print("this IS NOT expected if you have uploaded a zip file of short audios. Please check your file structure or make sure your audio language is supported.")
with open(os.path.join(filelists, 'esd.list'), 'w', encoding='utf-8') as f:
    for line in speaker_annos:
        f.write(line)

print("标注完成")

In [ ]:
# @title ### 配置文件生成
log_interval = 50 # @param {type:"integer"}
eval_interval = 100 # @param {type:"integer"}
epochs = 100 # @param {type:"integer"}
batch_size = 4 # @param {type:"integer"}
bert_device = "cuda" # @param ["cuda", "cpu"]
bert_processs = 2 # @param {type:"integer"}

import json
with open('configs/config.json') as fp:
  configs = json.load(fp)

configs['train']['log_interval'] = log_interval
configs['train']['eval_interval'] = eval_interval
configs['train']['epochs'] = epochs
configs['train']['batch_size'] = batch_size

configs['data']['training_files'] = os.path.join(filelists, 'train.list')
configs['data']['validation_files'] = os.path.join(filelists, 'val.list')
configs['data']['n_speakers'] = 1
configs['data']['spk2id'] = {speaker : 0}

configs_path = os.path.join(speaker_dir, 'config.json')
with open(configs_path, 'w') as fp:
  json.dump(configs, fp, indent=4)
  print(f'生成配置文件: {configs_path}')

configs = None
import yaml
with open('default_config.yml') as fp:
  configs = yaml.load(fp, Loader=yaml.FullLoader)

configs['dataset_path'] = speaker_dir
configs['preprocess_text']['transcription_path'] = 'filelists/esd.list'
configs['bert_gen']['device'] = bert_device
configs['bert_gen']['num_processes'] = bert_processs

with open('config.yml', 'w') as fp:
  yaml.dump(configs, fp)
  print(f'生成配置文件: config.yml')

In [ ]:
#@title 音频重采样
!python resample.py

In [ ]:
#@title 生成训练集、验证集
!python preprocess_text.py

In [ ]:
#@title 生成Bert
!python bert_gen.py

# Tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir $models_dir

# 训练

In [ ]:
!python train_ms.py

# 推理

In [ ]:
#@title 设置推理模型
model = "G_500.pth" # @param {type:"string"}

configs = None
import yaml
with open('config.yml') as fp:
  configs = yaml.load(fp, Loader=yaml.FullLoader)
  configs['webui']['model'] = f'models/{model}'
  configs['webui']['share'] = True

with open('config.yml', 'w') as fp:
  yaml.dump(configs, fp)
  print(f'更新配置文件: config.yml')


## WebUI推理

In [ ]:
#@title 启动WebUI
print('启动webui')
!python webui.py

In [ ]:
#@title 修复缺少frpc_linux_amd64提示
!wget https://cdn-media.huggingface.co/frpc-gradio-0.2/frpc_linux_amd64
!chmod +x frpc_linux_amd64
!mv frpc_linux_amd64 /usr/local/lib/python3.10/dist-packages/gradio/frpc_linux_amd64_v0.2

## 直接推理

In [ ]:
#@title 载入模型
import gc
import os
import logging
import re_matching
from tools.sentence import split_by_language

import scipy
import torch
from config import config
from infer import infer, latest_version, get_net_g, infer_multilang
import gradio as gr
import utils
from IPython.display import Audio

net_g = None
device = 'cuda'
hps = None

def free_up_memory():
    # Prior inference run might have large variables not cleaned up due to exception during the run.
    # Free up as much memory as possible to allow this run to be successful.
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

def process_mix(slice):
    _speaker = slice.pop()
    _text, _lang = [], []
    for lang, content in slice:
        content = content.split("|")
        content = [part for part in content if part != ""]
        if len(content) == 0:
            continue
        if len(_text) == 0:
            _text = [[part] for part in content]
            _lang = [[lang] for part in content]
        else:
            _text[-1].append(content[0])
            _lang[-1].append(lang)
            if len(content) > 1:
                _text += [[part] for part in content[1:]]
                _lang += [[lang] for part in content[1:]]
    return _text, _lang, _speaker

def process_auto(text):
    _text, _lang = [], []
    for slice in text.split("|"):
        if slice == "":
            continue
        temp_text, temp_lang = [], []
        sentences_list = split_by_language(slice, target_languages=["zh", "ja", "en"])
        for sentence, lang in sentences_list:
            if sentence == "":
                continue
            temp_text.append(sentence)
            if lang == "ja":
                lang = "jp"
            temp_lang.append(lang.upper())
        _text.append(temp_text)
        _lang.append(temp_lang)
    return _text, _lang

def format_utils(text, speaker):
    _text, _lang = process_auto(text)
    res = f"[{speaker}]"
    for lang_s, content_s in zip(_lang, _text):
        for lang, content in zip(lang_s, content_s):
            res += f"<{lang.lower()}>{content}"
        res += "|"
    return "mix", res[:-1]

def generate_audio(
    slices,
    sdp_ratio=0.5,
    noise_scale=0.5,
    noise_scale_w=0.9,
    length_scale=1.0,
    speaker="paimeng",
    language="mix",
    reference_audio=None,
    emotion="Happy",
    style_text=None,
    style_weight=0.7,
    skip_start=False,
    skip_end=False,
):
    audio_list = []
    # silence = np.zeros(hps.data.sampling_rate // 2, dtype=np.int16)
    global hps, device, net_g
    free_up_memory()

    with torch.no_grad():
        for idx, piece in enumerate(slices):
            skip_start = idx != 0
            skip_end = idx != len(slices) - 1
            audio = infer(
                piece,
                reference_audio=reference_audio,
                emotion=emotion,
                sdp_ratio=sdp_ratio,
                noise_scale=noise_scale,
                noise_scale_w=noise_scale_w,
                length_scale=length_scale,
                sid=speaker,
                language=language,
                hps=hps,
                net_g=net_g,
                device=device,
                skip_start=skip_start,
                skip_end=skip_end,
                style_text=style_text,
                style_weight=style_weight,
            )
            audio16bit = gr.processing_utils.convert_to_16_bit_wav(audio)
            audio_list.append(audio16bit)
    return audio_list

def generate_audio_multilang(
    slices,
    sdp_ratio,
    noise_scale,
    noise_scale_w,
    length_scale,
    speaker,
    language,
    reference_audio,
    emotion,
    skip_start=False,
    skip_end=False,
):
    audio_list = []
    # silence = np.zeros(hps.data.sampling_rate // 2, dtype=np.int16)

    free_up_memory()

    with torch.no_grad():
        for idx, piece in enumerate(slices):
            skip_start = idx != 0
            skip_end = idx != len(slices) - 1
            audio = infer_multilang(
                piece,
                reference_audio=reference_audio,
                emotion=emotion,
                sdp_ratio=sdp_ratio,
                noise_scale=noise_scale,
                noise_scale_w=noise_scale_w,
                length_scale=length_scale,
                sid=speaker,
                language=language[idx],
                hps=hps,
                net_g=net_g,
                device=device,
                skip_start=skip_start,
                skip_end=skip_end,
            )
            audio16bit = gr.processing_utils.convert_to_16_bit_wav(audio)
            audio_list.append(audio16bit)
    return audio_list

def process_text(
    text: str,
    speaker='paimeng',
    sdp_ratio=0.5,
    noise_scale=0.6,
    noise_scale_w=0.9,
    length_scale=1.0,
    language='mix',
    reference_audio=None,
    emotion='Happy',
    style_text=None,
    style_weight=0,
):
    audio_list = []
    if language == "mix":
        bool_valid, str_valid = re_matching.validate_text(text)
        if not bool_valid:
            return str_valid, (
                hps.data.sampling_rate,
                np.concatenate([np.zeros(hps.data.sampling_rate // 2)]),
            )
        for slice in re_matching.text_matching(text):
            _text, _lang, _speaker = process_mix(slice)
            if _speaker is None:
                continue
            print(f"Text: {_text}\nLang: {_lang}")
            audio_list.extend(
                generate_audio_multilang(
                    _text,
                    sdp_ratio,
                    noise_scale,
                    noise_scale_w,
                    length_scale,
                    _speaker,
                    _lang,
                    reference_audio,
                    emotion,
                )
            )
    elif language.lower() == "auto":
        _text, _lang = process_auto(text)
        print(f"Text: {_text}\nLang: {_lang}")
        audio_list.extend(
            generate_audio_multilang(
                _text,
                sdp_ratio,
                noise_scale,
                noise_scale_w,
                length_scale,
                speaker,
                _lang,
                reference_audio,
                emotion,
            )
        )
    else:
        audio_list.extend(
            generate_audio(
                text.split("|"),
                sdp_ratio,
                noise_scale,
                noise_scale_w,
                length_scale,
                speaker,
                language,
                reference_audio,
                emotion,
                style_text,
                style_weight,
            )
        )
    return audio_list

def init_model():
    global hps, net_g, device
    hps = utils.get_hparams_from_file(config.webui_config.config_path)
    # 若config.json中未指定版本则默认为最新版本
    version = hps.version if hasattr(hps, "version") else latest_version
    model_path = os.path.join(models_dir, model)
    print('load model: ' + model_path)
    net_g = get_net_g(model_path=model_path, version=version, device=device, hps=hps)

def text_to_audio(text, speaker, sdp_ratio, noise_scale, noise_scale_w, length_scale, language, filename):
    audios = process_text(text, speaker, sdp_ratio, noise_scale, noise_scale_w, length_scale, language, None, 'Happy')
    if len(audios) == 1:
        scipy.io.wavfile.write(filename, 44100, audios[0])
    else:
        print('生成失败')

init_model()

In [ ]:
#@title 直接推理
text = "大家好，这里是孙博士研究所！" # @param {type:"string"}
sdp_ratio = 0.5 # @param {type:"slider", min:0, max:1, step:0.1}
noise_scale = 0.6 # @param {type:"slider", min:0, max:1, step:0.1}
noise_scale_w = 0.9 # @param {type:"slider", min:0, max:1, step:0.1}
length_scale =  1.0 # @param {type:"slider", min:0.1, max:2, step:0.1}
language = "ZH" # @param ["ZH", "JP", "EN", "mix", "auto"]
filename = "output.wav" # @param {type:"string"}

text_to_audio(text,
              speaker,
              sdp_ratio,
              noise_scale,
              noise_scale_w,
              length_scale,
              language,
              os.path.join('/content', filename))

In [ ]:
#@title 播放音频
Audio(filename=os.path.join('/content', filename))

# 训练数据保存&恢复

In [ ]:
#@title 训练数据保存路径
data_save_dir = "/content/drive/MyDrive/data/save" # @param {type:"string"}

In [ ]:
#@title 恢复训练数据
from google.colab import drive
drive.mount('/content/drive')

print('删除Data目录')
!rm -fr {data_dir}
print('恢复训练数据中...')
!rsync -r --info=progress2 --info=name0 {data_save_dir}/Data /content/Bert-VITS2
!cp {data_dir}/config.yml /content/Bert-VITS2/config.yml
print('恢复完成')

In [ ]:
#@title 保存训练数据
from google.colab import drive
drive.mount('/content/drive')

print('模型保存中...')
!cp /content/Bert-VITS2/config.yml {data_dir}
!rsync -r --info=progress2 --info=name0 {data_dir} {data_save_dir}
print('保存完成！')

## 导出ONNX模型

In [ ]:
#@title 导出
from onnx_modules import export_onnx
import os

export_model = "G_500.pth" # @param {type:"string"}
Extra = "chinese"  # @param ["chinese", "japanese"]
model_path = os.path.join(models_dir, export_model)
config_path = os.path.join(speaker_dir, 'config.json')
novq = False
dev = False
print(model_path)

if not os.path.exists("onnx"):
    os.makedirs("onnx")
if not os.path.exists(f"onnx/{speaker}"):
    os.makedirs(f"onnx/{speaker}")
export_onnx(speaker, model_path, config_path, novq, dev, Extra)

In [ ]:
#@title 保存ONNX模型到网盘
onnx_save_dir = "/content/drive/MyDrive/data/onnx_model" # @param {type:"string"}
print('模型保存中...')
onnx_model_dir = "/content/Bert-VITS2/onnx"
!rsync -r --info=progress2 --info=name0 {onnx_model_dir} {onnx_save_dir}
print('保存完成！')